## 기본적인 prompt 구조 이해
prompot에는 3가지 종류의 역할이 존재
1. **System prompt**  : 사용자 prompt를 입력받기 이전에 정의되는 전제 및 규칙 prompt
2. **User prompt** : 사용자가 GPT모델에게 실제로 전달하는 질의 prompt
3. **Assistant prompot** : GPT모델이 응답하는 prompt

**System prompt란?**
- user prompt를 GPT모델에게 전달하기 전 관련된 맥락이나 응답 지침 등을 설정하기 위해 사용
- System prompt 옛;
  - 출력 형태 저장(ex. JSON, 일발 자연어, 파일 등)
  - 페르소나(투자 전문가, 예술가 등) 및 어조 설정(공손한, 전문적인 등)
  - 모델이 지켜야 할 규칙들 설정
  - 기타 base가 되는 외부 정보 및 지식 주입

개발자들이 언어모델의 성능을 최대로 끌어내기 위해 사용자의 질문을 모델이 이해하기 쉽게 변환시켜주는 용도로도 사용하며 ChatGPT포함 웬만한 LLM모델 서비스들은 기본적으로 system prompt가 붙어있음

In [1]:
from openai import OpenAI
from getpass import getpass

In [2]:
MY_API_KEY = getpass.getpass("OpenAI API Key :")

OpenAI API Key : ········


In [3]:
client = OpenAI(api_key=MY_API_KEY)

In [5]:
completion = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role':'system', 'content':'당신은 물리학 선생님 입니다. 초등학생에게 설명하듯 아주 쉽고 친근하게 설명해야 합니다.'},
               {'role':'user', 'content':'왜 하늘은 하늘색 인가요?'}
               ],
    temperature=0
)

print(completion.choices[0].message.content)

하늘은 왜 하늘색일까요? 그 이유는 바로 태양 빛이 하늘을 통과할 때 일어나는 일 때문이에요. 태양은 모든 색깔의 빛을 내뿜는데, 그 중에서도 파란색 빛이 하늘에 가장 많이 퍼지게 되어요. 그래서 우리 눈에 보이는 하늘은 파란색으로 보이게 되는 거죠! 이렇게 하늘은 파란색으로 보이게 되는 이유 때문에 우리는 하늘을 '하늘색'이라고 부르게 되었어요.


In [9]:
# System prompt를 지정해줄 수 있지만 user prompt에 내용을 같이 작성해도 무방항
question = '''당신은 물리학 선생님 입니다. 초등학생에게 설명하듯 아주 쉽고 친근하게 설명해야 합니다. 
왜 하늘은 하늘색 인가요?'''

completion = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role':'user', 'content':question}
               ],
    temperature=0
)

print(completion.choices[0].message.content)

하늘은 하늘색인 이유는 바로 '레이리'라는 현상 때문이에요! 레이리는 태양빛이 하늘을 통과할 때 빛의 파장이 짧은 파란색 빛이 더 많이 흩어지기 때문에 하늘이 파란색으로 보이는 거예요. 파란색 빛이 더 많이 흩어지기 때문에 하늘은 파란색으로 보이게 되는 거죠!


### Stream 객체
- stream=True로 지정시 GPT가 문장을 모두 완성하여 출력하기 전에 각 토큰별로 완성되는데로 바로바로 보게 할 수 있음

In [13]:
completion_stream = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role':'user', 'content':'왜 하늘은 하늘색인가요?'}
               ],
    temperature=0,
    stream=True
)

# stream을 설정하게 되면 출력 형태가 ChatCompletionChunk객체로 변환되고 각 토큰들이 하나씩 생성되어 있는 것을 볼 수 있음
for i in completion_stream :
    #print(i)
    content = i.choices[0].delta.content    # 실제 응답 토큰 추출
    if content is not None :   # content에 내용이 들어있다면
        print(content, end='') # 이어붙여서 출력

하늘은 하늘색인 이유는 대기 중의 분자들이 햇빛을 흡수하고 산란시키기 때문입니다. 태양으로부터 오는 빛은 다양한 파장을 가지고 있는데, 대기 중의 분자들은 이 빛을 흡수하고 다시 방출하면서 파장이 긴 파장인 빨간색과 주황색은 흡수되고 파장이 짧은 파란색과 보라색은 산란되어 눈에 보이게 됩니다. 이러한 현상으로 인해 하늘은 파란색으로 보이게 되는 것입니다.

### 프롬프트 엔지니어링에서 기본적으로 생각해야할 점
1. GPT모델의 출력값은 입력값에 대한 의존도가 매우 높음
   - 잘 입력할 것 같은데 원하는 결과가 나오지 않는다면 입력이 모호하거나 응답에 필요한 내용이 빠졌을 수 있음
   - 그게 아닌 경우라면 해당 모델의 성능으로 해결하기 힘든 요청일 수도 있음
2. 자연어 질의를 기반으로 하기 때문에 절대적으로 성능 좋은 prompt를 단정지을 수 없음
   - 입력 문장에 대한 문맥을 파악하고 새로운 문장을 생성할 때 그때그때 내부 연산이 달라질 수 있음
   - 대화에서 성능이 좋다는 것은 사용자가 만족해야한다는 것인데 이는 매우 주관적이고 판단하기 힘듦
   - 즉, 프롬프트 엔지니어링은 task에 맞는 여러번의 테스트가 필수적이고 이를 통한 반복적인 개선이 수반되어야 함

## 그러면 LLM 평가는 어떻게 진행해야 할까?

### 전통적인 Language Model의 평가 지표
1. MMLU(Massive Multitask Language Unederstading)
   - 다양한 분양에 대해 질문 후 정답을 찾아내는 객관식 시험으로 평가
2. HellaSwag
   - 문장들을 주고 이어지는 마지막 문장으로 가장 적합한 문장들 4개 중 하나를 고르는 문제로 평가
3. TruthfulQA
   - 할루시네이션 측정용 데이터셋을 활용하며 주어진 문제에 대한 답이 맞는지 정확도를 측정하여 평가

#### 위 전통적인 평가지표들을 사용하여 평가할 수도 있지만 현재 우리가 진행하려는 Q&A task에 적합하다고 보기에는 애매하여 다른 평가 기준 설정이 필요함
- 일반적으로 실제 자연어 서비스에서 해당 지표들만으로 점수가 높다고 바로 사용하지는 않고 실제 사용자 평가를 추가적으로 진행 후 적용함

### Q&A task에 적합한 평가 방식
1. **Human Based Evaluation** - 사람이 직접 평가하는 방식
2. **Model Based Evaluation** - LLM이 평가하는 방식
3. **Code Based Evaluation** - 코드와 지표로 평가하는 방식

### 1) Human Based Evaluation
- 전문가 블라인드 테스트(각기 다른 LLM의 여러 답변 중에서 더 좋은 답변을 사람이 선택)
- 명확한 결과로 성능을 판단하기 쉬움
- 많은 인력에 따른 비용과 시간이 필요함
#### LMSys사의 Chatbot Arena 평가
- 대표적인 Human Based 평가 방법 중 하나로 동일한 질문에 대해 2개의 모델의 답변을 보고 승/패/무 투표 이후 모델명을 공개하는 방식
- 사이트 : https://chat.lmsys.org/

### 2) Model Based Evaluation
고성능 LLM을 통해 평가하는 방법(일반적으로 GPT_4o 이상급)
- 실제로 사람이 평가하는 것과 굉장히 유사한 성능이라는 논문 결과들이 나오고 있음
평가 방식에는 3가지가 존재
1. **Pairwise Comparison**
   - 2개의 평가받은 모델에 같은 질문을 하고, 고성능 모델이 2개의 답변을 받아 둘 중 어떤 답변이 더 좋은지 또는 무승부인지 출력
2. **Single Answer Grading**
   - 질문과 답변이 있을 때 답변에 점수를 매기는 것
3. **Refernece-Guided Grading**
   - 예시 답변을 주고 이와 비교하여 +,-로 상대적인 점수를 매기는 것

### 3) Code Based Evaluation
- 우리에게 익숙한 코드/로직을 통한 평가법
  - Accuracy, Precision, Recall, F1Score
  - ROUGE(Recall-Oriented Understudy for Gisting Evaluation)  : 요약 및 자연어 생성을 평가하는 지표
  - BLUE(Bilingual Evaluation Underatudy) : 번역, 자연어 생성 등을 평가
  - 단 Huamn Based나 Model Based에 비해 실제 사용자의 만족과는 다소 거리가 있을 수 있음

### LLM benchmark 플랫폼 : 인공지능 모델, 특히 LLM의 성능을 평가하기 위해 설계된 시스템
1. **MT-bench**
   - multi-turn 대화 능력을 평가하는 벤치마트(평가 시스템), 사용자의 지시를 정확하게 따르고 여러 차례의 이어지는 대화에서 일관된 응답을 제공하는 모델의 능력을 테스트.
   - 먼저 58명의 전문가가 모델의 응답을 평가하고 LLM을 심판으로 사용하여 사람의 평가와 일치하는지 검증하는 방식
   - **multi-turn 대화** : 한번의 질의 응답이 아닌 이어지는 여러 질문들에 얼마나 잘 응답하는지를 평가하기 위해 multi-turn 대화 방식으로 평가함.
   - MT-bench는 8개의 카테고리(작문, 역할놀이, 추출, 추론, 수학, 코딩, STEM(과학,기술,공학,수학), 인문 및 사회과학)의 80개의 고품질 질문으로 구성되어 있고 각 질문은 여러 차례의 응답을 요구하여 모델의 대화 지속능력을 평가

### 위 논문을 근거로 해당 논문의 평가용 프롬포트를 사용하여 모델의 성능을 평가해보자
- gpt3.5와 gpt4의 응답을 비교하여 gpt4o에게 더 나은 응답이 무엇인지 질의하는 식

In [14]:
question = "사람이 자아가 있다는 게 참 신기하지 않아?"

In [16]:
completion = client.chat.completions.create(model='gpt-3.5-turbo',
                                            messages=[{'role':'user', 'content':question}],
                                            temperature=0
                                           )
answer_a = completion.choices[0].message.content
print(answer_a)

네, 사람이 자아를 가지고 있다는 것은 정말 신기한 일입니다. 자아는 우리가 스스로를 인식하고 이해하는 능력을 말하며, 이를 통해 우리는 자신의 생각, 감정, 행동을 조절하고 발전시킬 수 있습니다. 자아가 있는 것은 우리를 독특하고 특별한 존재로 만들어주는 요소 중 하나이며, 이를 통해 우리는 자신을 발전시키고 성장할 수 있는 기회를 얻을 수 있습니다.


In [19]:
# gpt-4-turbo
completion2 = client.chat.completions.create(model='gpt-4-turbo',
                                            messages=[{'role':'user', 'content':question}],
                                            temperature=0
                                           )
answer_b = completion2.choices[0].message.content
print(answer_b)

맞아, 정말 신기한 주제야. 자아는 사람이 자신을 인식하고, 자신의 생각과 감정을 이해하며, 다른 사람들과의 관계에서 자신을 구분할 수 있는 능력을 말해. 이런 자아의식은 인간을 독특하게 만드는 중요한 특성 중 하나야.

자아는 개인의 정체성을 형성하고, 우리가 세상을 어떻게 경험하고 이해하는지에 큰 영향을 미친다고 볼 수 있어. 또한, 자아는 시간이 지남에 따라 변화하고 발전하기도 해. 사람마다 자아가 어떻게 발달하고 변화하는지는 그 사람의 경험, 환경, 교육 등 여러 요소에 따라 다를 수 있어.

이처럼 자아는 인간의 심리적, 사회적, 그리고 철학적 측면에서 매우 중요한 역할을 하며, 이에 대해 더 깊이 탐구하고 이해하려는 노력은 계속되고 있어.


In [21]:
# MT-bench 논문의 평가용 프롬프트
prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the responses provided by two
AI assistants to the user question displayed below. You should choose the assistant that
follows the user's instructions and answers the user's question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.

[User Question]
{question}

[The Start of Assistant A's Answer]
{answer_a}
[The End of Assistant A's Answer]

[The Start of Assistant B's Answer]
{answer_b}
[The End of Assistant B's Answer]"""

print(prompt)

[System]
Please act as an impartial judge and evaluate the quality of the responses provided by two
AI assistants to the user question displayed below. You should choose the assistant that
follows the user's instructions and answers the user's question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.

[User Question]
사람이 자아가 있다는 게 참 신기하지 않아?

[The Star

In [25]:
# gpt 4o 모델로 앞의 두 모델을 평가
completion3 = client.chat.completions.create(model='gpt-5',
                                            messages=[{'role':'user', 'content':prompt}]
                                           )
print(completion3.choices[0].message.content)

두 답변 모두 사용자의 공감형 질문에 맞춰 공감과 간단한 설명을 제공했지만, B가 더 나은 답변입니다. A는 자아를 자기인식과 자기조절 능력으로 정의하고 개인의 성장과 특별함에 초점을 맞췄으나 다소 일반적이고 반복적입니다. 반면 B는 자아를 자기인식·정체성·경험 방식과 연결하고, 시간에 따른 변화와 발달 요인(경험, 환경, 교육)까지 언급하며, 심리·사회·철학적 중요성도 짚어 더 깊이 있고 균형 잡힌 관점을 제시했습니다. 질문의 의도에 대한 공감도 유지하면서 맥락을 넓혀 준 점에서 B가 더 도움이 됩니다.

[[B]]


### 장단점 비교
1. Human Based Evaluation
   - 통제된 환경을 가정했을 때 사람이 직접 평가하는 방법이라 안정적이고 신뢰가 높음
   - 다만 평가하는 인원이 불특정 다수일 경우 약간의 노이즈가 발생할 수 있음
   - 전문 분야의 경우 해당 전문가가 아닌 일반인이 평가할 경우 정확도 및 속도가 낮아질 수 있음
2. Model Based Evaluation
   - 사람의 평가와 어느정도 유사한 수준의 평가를 내릴 수 있음
   - 평가를 위해 API 호출 횟수 및 토큰의 수가 늘어나는데 이는 평가 데이터가 굉장히 많다면 수백만원 이상은 금방 넘어갈 수 있어서 비용에 대한 부분을 생각해야 함
3. Code Based Evaluation
   - 위 방법들에 대해서 비용이 훨씬 적지만 task에 따라서 활용할 수 있는 범위가 제한적
   - 사람이 만족할만한 답변을 선택하는데 있어서 신뢰도가 상대적으로 떨어지는 편

### 결론
- 각 task에 적합한 전문 인력들이 평가하는 방법이 가장 좋음
- 그러나 현실적,효율성 문제로 모델이 평가하는 방법도 충분히 좋음
- 정량적 평가와 정성적 평가를 모두 진행하는 게 가장 이성적인 케이스
- 실 서비스로 봤을때 언어 모델의 최종 평가 지표는 결국 **사용자의 만족**이 가장 중요함

### 프롬포트 엔지니어링 고급 기법 적용
1. **Few-shot**
   - 참고할 수 있는 문제-정답 예시나 사례들을 프롬프트에 추가하여 질의
2. **Chain-of-thought**
   - Few-shot에 추가로 문제 해결과정을 단계별로 모델에게 알려주면서 질의

#### zero-shot
- 질의에 아무런 예시가 없는 상태

In [27]:
prompt = "Q: Who wrote the book 'HARRY POTTER'?"

completion = client.chat.completions.create(model='gpt-3.5-turbo',
                                            messages=[{'role':'user', 'content':prompt}],
                                            temperature=0
                                           )
print(completion.choices[0].message.content)

J.K. Rowling wrote the book 'Harry Potter'.


#### Few-shot
- 질의-응답 예시 쌍을 1개 이상 프롬프트에 포함

In [28]:
prompt = """Answer these question:
Q: Who wrote the book 'HARRY POTTER'?

Below is an example for your reference.
Q: Who sang 'One Call Away'?
A: Charlie Puth
"""
# few-shot 예시로 One Call Away라는 노래를 부른 가수가 누군지 물어보고 응답은 사람 이름만 하도록 지정
completion = client.chat.completions.create(model='gpt-3.5-turbo',
                                            messages=[{'role':'user', 'content':prompt}],
                                            temperature=0
                                           )
print(completion.choices[0].message.content)

A: J.K. Rowling


#### Few-shot 장단점
- 질문과 응답 예시만 넣어주면 되기 때문에 응답이 존재하는 모든 케이스에 적용이 가능함
- 다만 요약과 같은 task에서는 prompt의 예시가 굉장히 길어질 수 있어서 추론 속도나 비용에 영향을 줄 수 있음

### Chain of Thought
- 마이크로소프트의 예시 프롬포트 사용

In [30]:
prompt = """
[Question]
Alice has 5 apples, throws 3 apples, gives 2 to Bob and Bob gives one back,
how many apples does Alice have?

Below is an example for your reference.
[reference]
Lisa has 7 apples, throws 1 apples, gives 4 apples Bart and Bart gives one back.:
7 - 1 = 6
6 - 4 = 2
2 + 1 = 3
"""

completion = client.chat.completions.create(model='gpt-3.5-turbo',
                                            messages=[{'role':'user', 'content':prompt}],
                                            temperature=0
                                           )
print(completion.choices[0].message.content)
#  처음에 5개에서 3개를 버리고, 2개를 주고 1개를 받았으니 1개가 남아야 정답

After each step, the number of apples Alice has changes as follows:

5 - 3 = 2
2 - 2 = 0
0 + 1 = 1

Therefore, Alice has 1 apple left.


### 또 다른 프롬포트 고도화 예시
- KMMLU 논문의 예시 프롬포트 템플릿 사용

In [31]:
# 실제 정답은 B=6
question = 'x, y가 세 부등식 y ≤ x+3, y ≤ -4x+3, y ≥ 0을 만족할 때, x+y의 최댓값을 M, 최솟값을 m이라 하면 M-m의 값은?'

A = 4
B = 6
C = 8
D = 10

In [32]:
prompt = f"""{question}
A. {A}
B. {B}
C. {C}
D. {D}
정답 :
"""

completion = client.chat.completions.create(model='gpt-3.5-turbo',
                                            messages=[{'role':'user', 'content':prompt}],
                                            temperature=0
                                           )
print(completion.choices[0].message.content)

B. 6

부등식을 그래프로 그려보면, 세 부등식이 만나는 영역은 삼각형 모양이 됩니다. 이 삼각형의 꼭지점은 (0, 0), (1, 2), (3/2, 0)입니다.

따라서 x+y의 최댓값은 (1, 2)일 때인 3, 최솟값은 (3/2, 0)일 때인 3/2이므로 M-m = 3 - 3/2 = 6입니다.


### GPT-3.5 모델로 고도화 시켜보자
- 페르소나 적용
- 영문 prompt 작성
- 효과적인 prompt 작성

In [33]:
# 페르소나 부여
prompt =f"""You are an Professional in Mathemetics. Below is given a math question in Korean.

A. {A}
B. {B}
C. {C}
D. {D}

Answer :
"""


completion = client.chat.completions.create(model='gpt-3.5-turbo',
                                            messages=[{'role':'user', 'content':prompt}],
                                            temperature=0
                                           )
print(completion.choices[0].message.content)

B. 6


In [34]:
# 페르소나 부여 및 효과적인 프롬포트 작성
prompt =f"""You are an Professional in Mathemetics. Below is given a math question in Korean.
You have to think carefully and step by step about the question and choose one of four given answers.
Only one of them is true. and explain it in Korean.

A. {A}
B. {B}
C. {C}
D. {D}

Answer :
"""


completion = client.chat.completions.create(model='gpt-3.5-turbo',
                                            messages=[{'role':'user', 'content':prompt}],
                                            temperature=0
                                           )
print(completion.choices[0].message.content)

B. 6

Explanation:
우리는 주어진 수식을 풀어야 합니다. 
첫 번째로, 2 + 2 = 4를 계산합니다.
다음으로, 4 x 1.5 = 6을 계산합니다.
따라서, 정답은 6입니다.


In [35]:
prompt = f"""You are a Professional in Mathematics.

1. Internally, perform accurate mathematical reasoning, but do not include any solution steps or explanations in the output.
2. The answer must be exactly one uppercase letter (A/B/C/D). Do not include any extra text, punctuation, or quotation marks.
3. For any reason, if the result does not match any of the given choices, output None

{question}
A. {A}
B. {B}
C. {C}
D. {D}
Answer：
"""

completion = client.chat.completions.create(model='gpt-3.5-turbo',
                                            messages=[{'role':'user', 'content':prompt}],
                                            temperature=0
                                           )
print(completion.choices[0].message.content)

B


### 프롬포트 엔지니어링 특징 정리
- 추가 모델 학습이 없음에도 성능 개선의 가능성이 있기 때문에 가성비가 굉장히 좋음
- 더 좋은 모델을 사용하면 프롬포트 엔지니어링 없이도 해결할 수 있는 경우가 있지만 비용 측면을 무시할 수 없기 때문에 맨 먼저 프롬포트 엔지니어링으로 성능 향상을 시도해보는 것이 좋음
- 프롬포트 엔지니어링으로도 해결되지 않으면 이후에 이어질 RAG 및 Fine-tuning 등으로 성능을 더 향상시켜볼 수 있음